In [7]:
import numpy as np
import pandas as pd

from ConvertCSV import DataReady
from FinRockDataFeeder import PdDataFeeder
from FinRockTradingEnv import TradingEnv
from FinRockRender import PygameRender
from FinRockScaler import ZScoreScaler
from FinRockReward import AccountValueChangeReward
from Indicators import BolingerBands, SMA, RSI, PSAR, MACD
from FinRockMetrics import AccountValue

In [8]:
df = pd.read_csv("/home/zymantas/Desktop/Training_data/5minData/ATOMUSDT5min.csv")

DataReady(df)

df = df[:-1000] # leave 1000 for testing

                timestamp    open    high     low   close   volume
0     2024-03-11 02:00:00  13.190  13.233  13.179  13.219  10036.8
1     2024-03-11 02:05:00  13.219  13.276  13.217  13.243  20072.7
2     2024-03-11 02:10:00  13.243  13.262  13.201  13.262   8939.3
3     2024-03-11 02:15:00  13.262  13.264  13.203  13.205   9896.9
4     2024-03-11 02:20:00  13.205  13.209  13.150  13.167   8269.5
...                   ...     ...     ...     ...     ...      ...
6529  2024-04-02 19:05:00  10.943  11.030  10.931  11.013  22089.6
6530  2024-04-02 19:10:00  11.013  11.031  10.992  11.031  16393.6
6531  2024-04-02 19:15:00  11.031  11.031  10.950  10.976  10379.1
6532  2024-04-02 19:20:00  10.976  10.976  10.920  10.959  12796.1
6533  2024-04-02 19:25:00  10.959  10.959  10.939  10.949   2230.7

[6534 rows x 6 columns]


In [9]:
pd_data_feeder = PdDataFeeder(
    df = df,
    indicators = [
        BolingerBands(data=df, period=20, std=2),
    ]
)

In [10]:
env = TradingEnv(
    data_feeder = pd_data_feeder,
    output_transformer = ZScoreScaler(),
    initial_balance = 1000.0,
    max_episode_steps = 1000,
    window_size = 5,
    metrics = [
        AccountValue()
    ]
)
action_space = env.action_space
input_shape = env.observation_space.shape

env.save_config()

In [11]:
pygameRender = PygameRender(frame_rate=5)

state, info = env.reset()
pygameRender.render(info)
rewards = 0.0
while True:
    # simulate model prediction, now use random action
    action = np.random.randint(0, action_space)

    state, reward, terminated, truncated, info = env.step(action)
    rewards += reward
    pygameRender.render(info)

    if terminated or truncated:
        print(info['states'][-1].account_value, rewards)
        rewards = 0.0
        state, info = env.reset()
        pygameRender.reset()